# A simple scheduler

If the function is just not doing anything, why do I have to keep the CPU idle?

Let's spin up a simple scheduler and let the scheduler know that CPU is idle. And the scheduler can then start by looking if there is any other work to do, if yes then do it. Simple.

But for the first take, let's keep sleep around and see how to interleave the calls

## Dissecting on what is a scheduler

A scheduler is a setup which can take in requests and schedule them at a fixed sequence. Think of it as an orchestrator. It takes in requests and queues them up for execution.

Observe the word queue, the scheduler is essentially a queue which maintains a list of executables and executes them. Python natively has `deque` for queuing, let's build a scheduler using `deque`.

In [1]:
# Scheduler
import time
from collections import deque


class Scheduler:
    def __init__(self):
        self.ready = deque()

    def call_soon(self, func):
        self.ready.append(func)

    def run(self):
        while self.ready:
            func = self.ready.popleft()
            func()


scheduler = Scheduler()

Now that the scheduler is in place, let's swap the implementation of `countdown` and `countup` to use the scheduler

count down is straight forward. after sleeping, ask the scheduler to queue up the next execution

In [2]:
def countdown(n):
    if n > 0:
        print("down", n)
        time.sleep(1)
        scheduler.call_soon(lambda: countdown(n - 1))

countup is interesting, think about it. countdown takes in a number and calls itself, so there is _no state to maintain_

countup needs to maintain two variables, one is where to stop (which comes from the argument) and the other is till how far it has counted. Where do we maintain this?

One way of solving it is by using closures. We can have an inner fn take in a counter argument and then use both vars as state to run the countup.

And finally kick it off with executing with zero.

In [3]:
def countup(stop):
    def _run(counter: int):
        if counter < stop:
            print("up", counter)
            time.sleep(1)
            scheduler.call_soon(lambda: _run(counter + 1))

    _run(0)

Piecing it all together

In [4]:
scheduler.call_soon(lambda: countdown(5))
scheduler.call_soon(lambda: countup(5))
scheduler.run()

down 5
up 0
down 4
up 1
down 3
up 2
down 2
up 3
down 1
up 4


Awesome, it works as expected!

But still, during sleep the thread is just idle! That is not acceptable, we'll need to change that now!

## Time sequencing

In [6]:
# TODO